In [1]:
import pandas as pd

df = pd.read_csv('vehicles.csv')

In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 26 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   url           426880 non-null  object 
 2   region        426880 non-null  object 
 3   region_url    426880 non-null  object 
 4   price         426880 non-null  int64  
 5   year          425675 non-null  float64
 6   manufacturer  409234 non-null  object 
 7   model         421603 non-null  object 
 8   condition     252776 non-null  object 
 9   cylinders     249202 non-null  object 
 10  fuel          423867 non-null  object 
 11  odometer      422480 non-null  float64
 12  title_status  418638 non-null  object 
 13  transmission  424324 non-null  object 
 14  VIN           265838 non-null  object 
 15  drive         296313 non-null  object 
 16  size          120519 non-null  object 
 17  type          334022 non-null  object 
 18  pain

In [3]:
df.drop(['id','url','region','region_url','cylinders','VIN','drive','size','paint_color','image_url','description','county','state','lat','long','posting_date'],inplace=True,axis=1)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         426880 non-null  int64  
 1   year          425675 non-null  float64
 2   manufacturer  409234 non-null  object 
 3   model         421603 non-null  object 
 4   condition     252776 non-null  object 
 5   fuel          423867 non-null  object 
 6   odometer      422480 non-null  float64
 7   title_status  418638 non-null  object 
 8   transmission  424324 non-null  object 
 9   type          334022 non-null  object 
dtypes: float64(2), int64(1), object(7)
memory usage: 32.6+ MB


In [5]:
df.isna().sum()

price                0
year              1205
manufacturer     17646
model             5277
condition       174104
fuel              3013
odometer          4400
title_status      8242
transmission      2556
type             92858
dtype: int64

In [6]:
df['transmission'] = df['transmission'].apply(lambda x: True if isinstance(x, str) and x.lower() == 'automatic' else False)
df['title_status'] = df['title_status'].apply(lambda x: True if isinstance(x, str) and x.lower() == 'clean' else False)
df['fuel'] = df['fuel'].apply(lambda x: True if isinstance(x, str) and x.lower() == 'gas' else False)

In [7]:
df.isna().sum()

price                0
year              1205
manufacturer     17646
model             5277
condition       174104
fuel                 0
odometer          4400
title_status         0
transmission         0
type             92858
dtype: int64

In [8]:
df.drop(['model','type','condition'], inplace=True,axis=1)

In [9]:
df.isna().sum()

price               0
year             1205
manufacturer    17646
fuel                0
odometer         4400
title_status        0
transmission        0
dtype: int64

In [10]:
df.nunique()

price            15655
year               114
manufacturer        42
fuel                 2
odometer        104870
title_status         2
transmission         2
dtype: int64

In [11]:
df.dropna(subset=['year','odometer'],inplace=True)

In [12]:
# Get the top 5 manufacturers by count
top_5_manufacturers = df['manufacturer'].value_counts().nlargest(10).index

# Replace all other manufacturers with 'Other'
df['manufacturer'] = df['manufacturer'].apply(lambda x: x if x in top_5_manufacturers else 'Other')

# Verify the changes
print(df['manufacturer'].value_counts())

manufacturer
Other        141178
ford          70313
chevrolet     54414
toyota        33790
honda         21026
nissan        18818
jeep          18814
ram           18227
gmc           16609
bmw           14609
dodge         13546
Name: count, dtype: int64


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 421344 entries, 27 to 426879
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         421344 non-null  int64  
 1   year          421344 non-null  float64
 2   manufacturer  421344 non-null  object 
 3   fuel          421344 non-null  bool   
 4   odometer      421344 non-null  float64
 5   title_status  421344 non-null  bool   
 6   transmission  421344 non-null  bool   
dtypes: bool(3), float64(2), int64(1), object(1)
memory usage: 17.3+ MB


In [14]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV
import numpy as np

df_final = pd.get_dummies(df, columns=['manufacturer'])
# 1. Split the data
X = df_final.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 2. Train and evaluate models
# Initialize models
lr_model = LinearRegression()
rf_model = RandomForestRegressor(random_state=42)

# Train models
lr_model.fit(X_train, y_train)
rf_model.fit(X_train, y_train)

# Make predictions
lr_pred = lr_model.predict(X_test)
rf_pred = rf_model.predict(X_test)

# Calculate metrics
def calculate_metrics(y_true, y_pred, model_name):
    mse = mean_squared_error(y_true, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_true, y_pred)
    
    print(f"\n{model_name} Metrics:")
    print(f"MSE: {mse:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"R²: {r2:.4f}")
    
    return mse, rmse, r2

# Calculate metrics for both models
lr_metrics = calculate_metrics(y_test, lr_pred, "Linear Regression")
rf_metrics = calculate_metrics(y_test, rf_pred, "Random Forest")

# 3. Optimize the better model using GridSearchCV
# Assuming Random Forest performs better (usually does for this type of data)
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

grid_search = GridSearchCV(
    RandomForestRegressor(random_state=42),
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    n_jobs=-1
)

grid_search.fit(X_train, y_train)

# Print best parameters and score
print("\nBest Parameters:")
print(grid_search.best_params_)

# Evaluate optimized model
best_model = grid_search.best_estimator_
best_pred = best_model.predict(X_test)
calculate_metrics(y_test, best_pred, "Optimized Random Forest")

# Feature importance for the optimized Random Forest
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': best_model.feature_importances_
})
print("\nTop 10 Most Important Features:")
print(feature_importance.sort_values('importance', ascending=False).head(10))


Linear Regression Metrics:
MSE: 23809964493175.24
RMSE: 4879545.52
R²: -0.0013

Random Forest Metrics:
MSE: 60436835259510.20
RMSE: 7774113.15
R²: -1.5417

Best Parameters:
{'max_depth': 10, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 100}

Optimized Random Forest Metrics:
MSE: 33714867947481.43
RMSE: 5806450.55
R²: -0.4179

Top 10 Most Important Features:
                   feature  importance
2                 odometer    0.570595
0                     year    0.212121
5       manufacturer_Other    0.094961
4             transmission    0.079180
1                     fuel    0.015865
15     manufacturer_toyota    0.009683
7   manufacturer_chevrolet    0.007102
12       manufacturer_jeep    0.005919
9        manufacturer_ford    0.003394
3             title_status    0.001131
